In [ ]:
!pip install simplet5
!pip install PyAudio
!pip install SpeechRecognition pydub

In [ ]:
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os
for dirname, _, filenames in os.walk('/content/drive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
r = sr.Recognizer()

In [ ]:
filename = "/content/machine-learning_speech-recognition_16-122828-0002.wav"
df = pd.read_csv('/content/drive/MyDrive/news_summary.csv', encoding='latin-1', usecols=['headlines', 'text'])

In [ ]:
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [ ]:
path = "/content/drive/MyDrive/machine-learning_speech-recognition_7601-291468-0006.wav"
a = get_large_audio_transcription(path)
print("\nFull text:", a)


audio-chunks/chunk1.wav : His abode which you had fixed in a bowery or country seat. 
audio-chunks/chunk2.wav : Have a short distance from the city. 
audio-chunks/chunk3.wav : Just at what is now called dutch street. 
audio-chunks/chunk4.wav : Soon abounded with proofs of his ingenuity. 
audio-chunks/chunk5.wav : Patent smokejack. 
audio-chunks/chunk6.wav : It required a horse to work some. 
audio-chunks/chunk7.wav : Dutch oven roasted meat without fire. 
audio-chunks/chunk8.wav : Carts that went before the horses. 
audio-chunks/chunk9.wav : Weather cox that turned against the wind and other wrongheaded contrivances. 
audio-chunks/chunk10.wav : So just understand confound it all beholders. 

Full text: His abode which you had fixed in a bowery or country seat. Have a short distance from the city. Just at what is now called dutch street. Soon abounded with proofs of his ingenuity. Patent smokejack. It required a horse to work some. Dutch oven roasted meat without fire. Carts that went b

In [ ]:
df=df.rename(columns={"headlines":"target_text", "text":"source_text"})
df = df[['source_text', 'target_text']]

In [ ]:
df['source_text']="summarize: " + df['source_text']

In [ ]:
df

,source_text,target_text
0,summarize: The Administration of Union Territo...,Daman & Diu revokes mandatory Rakshabandhan in...
1,summarize: Malaika Arora slammed an Instagram ...,Malaika slams user who trolled her for 'divorc...
2,summarize: The Indira Gandhi Institute of Medi...,'Virgin' now corrected to 'Unmarried' in IGIMS...
3,summarize: Lashkar-e-Taiba's Kashmir commander...,Aaj aapne pakad liya: LeT man Dujana before be...
4,summarize: Hotels in Maharashtra will train th...,Hotel staff to get training to spot signs of s...
...,...,...
4509,summarize: Fruit juice concentrate maker Rasna...,Rasna seeking ?250 cr revenue from snack categ...
4510,summarize: Former Indian cricketer Sachin Tend...,Sachin attends Rajya Sabha after questions on ...
4511,"summarize: Aamir Khan, while talking about rea...",Shouldn't rob their childhood: Aamir on kids r...
4512,summarize: The Maharashtra government has init...,"Asha Bhosle gets ?53,000 power bill for unused..."


In [ ]:
train_df, test_df=train_test_split(df, test_size=0.3)
train_df.shape, test_df.shape

((3159, 2), (1355, 2))

In [ ]:
from simplet5 import SimpleT5
model=SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
model.train(train_df=train_df[:5000],
eval_df=test_df[:100],
source_max_token_len=128,
target_max_token_len=58,

batch_size=8, max_epochs=2, use_gpu=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /content/lightning_logs

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (ResultMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)
Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
!mkdir outputs

mkdir: cannot create directory ‘outputs’: File exists


In [ ]:
!(cd outputs; ls )

simplet5-epoch-0-train-loss-1.5757-val-loss-1.1899
simplet5-epoch-1-train-loss-1.1606-val-loss-1.21


In [ ]:
model.load_model('t5',"/content/outputs/simplet5-epoch-1-train-loss-1.1606-val-loss-1.21", use_gpu=True)

In [ ]:
text_to_summarize= r"Russia's sanctions against Gazprom Germania and its subsidiaries could cost German taxpayers and gas users an extra 5 billion euros ($5.4 billion) a year to pay for replacement gas, the Welt am Sonntag weekly reported, citing industry representatives.In May, Russia decided to stop supplying Gazprom Germania, which had been the German subsidiary of Gazprom, after Berlin put the company under trustee management due to Russia's invasion of Ukraine. read moreSince then, the Bundesnetzagentur energy regulator, acting as trustee, has had to buy replacement gas on the market to fulfil supply contracts with German municipal utilities and regional suppliers.The economy ministry estimates an extra 10 million cubic meters per day are required, said a ministry spokesperson, confirming a number cited by the newspaper.The quantities are procured on the market and at market prices. No information can be given on the exact amounts due to commercial confidentiality,said the spokesperson in an emailed response.Welt am Sonntag said the current cost would be about 3.5 billion euros a year and that further costs could arise from the filling of the Rehden natural gas storage facility which Economy Minister Robert Habeck ordered on Wednesday, it said.The paper also said the additional costs would be passed on to energy suppliers and end customers in the form of a gas levy from October.The ministry spokesperson said supplies were not at risk."

In [ ]:
model.predict(text_to_summarize)

["'Welt am Sonntag' reported that Russia's sanctions against Gazprom Germania could cost taxpayers an extra 5 billion euros"]